In [ ]:
from data_transformations import mnist_transform
from data_loaders import mnist_dataloader
from loss_funcs import CrossEntropyLoss
from models import ConvNet
from trainers import Trainer, MPTrainer
from testers import Tester
import numpy as np

import matplotlib.pyplot as plt
from loggers import set_logger

In [ ]:
set_logger(data_name='mnist', save_path='./loggers/log')

In [ ]:
train_dataset, test_dataset = mnist_dataloader.get_dataset(
    './datasets', transform=mnist_transform())

print('Train data set:', len(train_dataset))
print('Test data set:', len(test_dataset))

In [ ]:
train_dataloader, valid_dataloader, test_dataloader = mnist_dataloader.loader(
    train_dataset, test_dataset)

train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

In [ ]:
loss_func = CrossEntropyLoss()

In [ ]:
model = ConvNet(num_classes=10).cuda()
model

In [ ]:
print("Total number of parameters =", np.sum(
    [np.prod(parameter.shape) for parameter in model.parameters()]))

In [ ]:
trainer = Trainer(model, train_dataloader=train_dataloader, valid_dataloader=valid_dataloader,
                  train_epochs=20, valid_epochs=2, learning_rate=0.001, loss_func=loss_func, optimization_method='Adam')

In [ ]:
model, losses, accuracies = trainer.run()

In [ ]:
_ = plt.plot(losses['train'], '-b', losses['val'], '--r')

In [ ]:
tester = Tester(model=model, test_dataloader=test_dataloader, use_gpu=True)

In [ ]:
tester.run()

In [ ]:
trainer.save_model('saved_models/cnn_mnist.model')

In [ ]:
import torch
import time

from models import LRPModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def plot_relevance_scores(
    x: torch.tensor, r: torch.tensor, name: str,) -> None:
    """Plots results from layer-wise relevance propagation next to original image.
    Method currently accepts only a batch size of one.
    Args:
        x: Original image.
        r: Relevance scores for original image.
        name: Image name.
    """
    max_fig_size = 20

    _, _, img_height, img_width = x.shape
    max_dim = max(img_height, img_width)
    fig_height, fig_width = (
        max_fig_size * img_height / max_dim,
        max_fig_size * img_width / max_dim,
    )

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(fig_width, fig_height))

    x = x[0].squeeze().permute(1, 2, 0).detach().cpu()
    x_min = x.min()
    x_max = x.max()
    x = (x - x_min) / (x_max - x_min)
    axes[0].imshow(x)
    axes[0].set_axis_off()

    r_min = r.min()
    r_max = r.max()
    r = (r - r_min) / (r_max - r_min)
    axes[1].imshow(r, cmap="afmhot")
    axes[1].set_axis_off()

    fig.tight_layout()
    plt.show()

In [ ]:
lrp_model = LRPModel(model=model, top_k=0.02)

for i, (x, y) in enumerate(test_dataloader):
    x = x.to(device)
    # y = y.to(device)  # here not used as method is unsupervised.

    t0 = time.time()
    r = lrp_model.forward(x)
    print("{time:.2f} FPS".format(time=(1.0 / (time.time() - t0))))

    plot_relevance_scores(x=x, r=r, name=str(i))